In [9]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
from numpy import array_equal

from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [10]:
events_dict = {
    'BLUE: first_blood': 0,
    'BLUE: dragon': 1,
    'BLUE: herald': 2,
    'BLUE: first_tower_top': 3,
    'BLUE: first_tower_mid': 4,
    'BLUE: first_tower_bot': 5,
    'BLUE: second_tower_top': 6,
    'BLUE: second_tower_mid': 7,
    'BLUE: second_tower_bot': 8,
    'BLUE: third_tower_top': 9,
    'BLUE: third_tower_mid': 10,
    'BLUE: third_tower_bot': 11,
    'BLUE: inhibitor_top': 12,
    'BLUE: inhibitor_mid': 13,
    'BLUE: inhibitor_bot': 14,
    'BLUE: baron': 15,
    'BLUE: elder_dragon': 16,
    'BLUE: nexus_tower': 17,
    'BLUE: nexus': 18,
    'RED: first_blood': 19,
    'RED: dragon': 20,
    'RED: herald': 21,
    'RED: first_tower_top': 22,
    'RED: first_tower_mid': 23,
    'RED: first_tower_bot': 24,
    'RED: second_tower_top': 25,
    'RED: second_tower_mid': 26,
    'RED: second_tower_bot': 27,
    'RED: third_tower_top': 28,
    'RED: third_tower_mid': 29,
    'RED: third_tower_bot': 30,
    'RED: inhibitor_top': 31,
    'RED: inhibitor_mid': 32,
    'RED: inhibitor_bot': 33,
    'RED: baron': 34,
    'RED: elder_dragon': 35,
    'RED: nexus_tower': 36,
    'RED: nexus': 37
}

df = pd.read_csv('data/clean-one-line.csv')
df = df.drop('game', axis=1)
data = df.values.tolist()

for game in data:
    for i, s in enumerate(game):
        if s in events_dict:
            game[i] = events_dict.get(s)

labelsArray = []

for game in data:
    labelsArray += game

sequence = [b for b in labelsArray if not(isinstance(b, float))]
n_in = 9

In [11]:
def splitSequence(sequence, n_steps_in, n_steps_out=1):
	X, y = [], []
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return X, y

In [12]:
x_train, x_test = train_test_split(sequence,test_size=0.20)
len(x_train)

8054

In [13]:
train, y_train = splitSequence(x_train, n_in)
print(train[:5])
y_train[:5]

[[5, 17, 37, 25, 34, 1, 13, 0, 24], [17, 37, 25, 34, 1, 13, 0, 24, 1], [37, 25, 34, 1, 13, 0, 24, 1, 1], [25, 34, 1, 13, 0, 24, 1, 1, 21], [34, 1, 13, 0, 24, 1, 1, 21, 15]]


[[1], [1], [21], [15], [36]]

In [14]:
test, y_test = splitSequence(x_test, n_in)
print(test[:5])
y_test[:5]

[[24, 32, 6, 23, 20, 24, 24, 14, 36], [32, 6, 23, 20, 24, 24, 14, 36, 20], [6, 23, 20, 24, 24, 14, 36, 20, 36], [23, 20, 24, 24, 14, 36, 20, 36, 29], [20, 24, 24, 14, 36, 20, 36, 29, 14]]


[[20], [36], [29], [14], [6]]

In [15]:
df_test = pd.DataFrame({'event_1': [],
                   'event_2': [],
                   'event_3': [],
                   'event_4': [],
                   'event_5': [],
                   'event_6': [],
                   'event_7': [],
                   'event_8': [],
                   'event_9': [],
                   'target': []})

df_train = pd.DataFrame({'event_1': [],
                   'event_2': [],
                   'event_3': [],
                   'event_4': [],
                   'event_5': [],
                   'event_6': [],
                   'event_7': [],
                   'event_8': [],
                   'event_9': [],
                   'target': []})

In [16]:
for index,test_row in enumerate(test):
    event_1 = test_row[0]
    event_2 = test_row[1]
    event_3 = test_row[2]
    event_4 = test_row[3]
    event_5 = test_row[4]
    event_6 = test_row[5]
    event_7 = test_row[6]
    event_8 = test_row[7]
    event_9 = test_row[8]
    df_test.loc[len(df_test.index)] = [event_1, event_2, event_3, event_4, event_5, event_6, event_7, event_8, event_9, y_test[index][0]]

In [17]:
y_test = df_test['target'].copy()
X_test = df_test.drop(['target'],axis=1)
df_test

,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,target
0,24.0,32.0,6.0,23.0,20.0,24.0,24.0,14.0,36.0,20.0
1,32.0,6.0,23.0,20.0,24.0,24.0,14.0,36.0,20.0,36.0
2,6.0,23.0,20.0,24.0,24.0,14.0,36.0,20.0,36.0,29.0
3,23.0,20.0,24.0,24.0,14.0,36.0,20.0,36.0,29.0,14.0
4,20.0,24.0,24.0,14.0,36.0,20.0,36.0,29.0,14.0,6.0
...,...,...,...,...,...,...,...,...,...,...
2000,18.0,7.0,25.0,5.0,24.0,33.0,13.0,0.0,35.0,32.0
2001,7.0,25.0,5.0,24.0,33.0,13.0,0.0,35.0,32.0,29.0
2002,25.0,5.0,24.0,33.0,13.0,0.0,35.0,32.0,29.0,36.0
2003,5.0,24.0,33.0,13.0,0.0,35.0,32.0,29.0,36.0,37.0


In [18]:
for index, train_row in enumerate(train):
    event_1 = train_row[0]
    event_2 = train_row[1]
    event_3 = train_row[2]
    event_4 = train_row[3]
    event_5 = train_row[4]
    event_6 = train_row[5]
    event_7 = train_row[6]
    event_8 = train_row[7]
    event_9 = train_row[8]
    df_train.loc[len(df_train.index)] = [event_1, event_2, event_3, event_4, event_5, event_6, event_7, event_8, event_9, y_train[index][0]]

In [19]:
y_train = df_train['target'].copy()
X_train = df_train.drop(['target'],axis=1)
y_train

0        1.0
1        1.0
2       21.0
3       15.0
4       36.0
        ... 
8040    32.0
8041     3.0
8042     1.0
8043    26.0
8044     3.0
Name: target, Length: 8045, dtype: float64

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression trained


c:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [21]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test), average='micro'),
    'Recall': recall_score(y_test,model.predict(X_test), average='micro'),
    'F1-Score': f1_score(y_test,model.predict(X_test), average='micro')
    })
scores = pd.DataFrame(scores_list)

In [22]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.089776,0.089776,0.089776,0.089776
1,Support Vector Machine (Linear Kernel),0.065835,0.065835,0.065835,0.065835
2,Support Vector Machine (RBF Kernel),0.089277,0.089277,0.089277,0.089277
3,Decission Tree,0.036908,0.036908,0.036908,0.036908
4,Adaboost,0.076309,0.076309,0.076309,0.076309
5,Random Forest,0.069327,0.069327,0.069327,0.069327
6,Gradient Boosting Classifier,0.063840,0.063840,0.063840,0.063840
